In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

This notebook is an initial draft at coding up the functions for replicating/altering the Commodity
Trade Matters model. This is the counterfactual setup given, it is altered such that the counterfactual
scenario is looking at the impact of a supply shock. This is captured by R_hat: the endowment of 
natural resources.

In order to have a consistent framework: axis 0 will default to countries, axis 1 will default to countries
and axis 2 will default to goods (commodities or final goods).

In [4]:
syn_data = {'g':4, 'k':4, 'n':3}

In [5]:
## Expenditure

In [6]:
def get_E_hat(w_hat, r_hat, data):
    ''' This is a function that returns the total expenditure change for all countries as a vector E_hat. 
    variables: w_hat, r_hat 
    parameters: data['e_L'], data['e_R']
    output: E_hat (nx1)
    '''
    E_hat = w_hat*data['e_L'] + (r_hat * data['e_R']) @ np.ones((data['g'], 1))
    return E_hat

In [7]:
syn_data['e_L'] = np.ones((3,1))*.5
syn_data['e_R'] = np.ones((3, 4))*2
w_hat = np.ones((3,1))
r_hat = np.ones((3, 4))

In [8]:
get_E_hat(w_hat, r_hat, syn_data)

array([[8.5],
       [8.5],
       [8.5]])

In [9]:
# Seems right

In [10]:
## Price index

In [11]:
def get_P_f_hat(P_f_goods_hat, data):
    ''' This is a function that returns the price index change for a country as a vector P_f_hat.
    variables: P_f_goods_hat
    parameters: data['alpha'], data['sigma']
    output: P_f_hat (nx1)
    '''
    new_matrix = data['alpha'] * (P_f_goods_hat)**(1-data['sigma']) 
    P_f_hat = (new_matrix @ np.ones((data['k'], 1)) )**(1/(1-data['sigma']))
    
    return P_f_hat

In [12]:
syn_data['sigma'] = 2 
syn_data['alpha']=np.ones((3, 4))*2

P_f_goods_hat = np.ones((3, 4))*2

In [13]:
get_P_f_hat(P_f_goods_hat, syn_data)

array([[0.25],
       [0.25],
       [0.25]])

In [14]:
# Seems right

In [15]:
def get_P_f_goods_hat(C_f_hat, data):
    ''' This is a function that returns the price index for final goods change for a country and final good matrix.
    variables: C_f_hat
    parameters: data['lambda_f'], data['theta_k']
    output: P_f_goods_hat (nxk)
    '''
    
    P_f_goods_hat = np.zeros((data['n'], data['k']))
    C_f_hat_temp = C_f_hat ** (-data['theta_k'].reshape((1, data['k'])))
    part1 = np.sum(data['lambda_f'] * C_f_hat_temp.reshape((1, data['n'], data['k'])), axis=1)
    part2 = part1.reshape((data['n'], data['k']))
    P_f_goods_hat = part2 ** (-1/data['theta_k'].reshape((1, data['k'])))

    return P_f_goods_hat

In [20]:
syn_data['lambda_f'] = np.ones((3, 3, 4))*2
syn_data['theta_k'] = np.linspace(1, 4, 4).reshape((4,1)) 

C_f_hat = np.ones((3, 4))*4

In [21]:
P_f_goods_hat = get_P_f_goods_hat(C_f_hat, syn_data)

In [22]:
P_f_goods_hat

array([[0.66666667, 1.63299316, 2.20128483, 2.55577242],
       [0.66666667, 1.63299316, 2.20128483, 2.55577242],
       [0.66666667, 1.63299316, 2.20128483, 2.55577242]])

In [29]:
get_P_f_hat(P_f_goods_hat, syn_data)

array([[0.16903749],
       [0.16903749],
       [0.16903749]])

In [23]:
def get_P_c_goods_hat(C_g_hat, data):
    ''' This is a function that returns the price index for change inn commodities for a country and commodity matrix.
    variables: C_g_hat
    parameters: data['lambda_c'], data['theta_g']
    output: P_g_goods_hat (nxg)
    '''
        
    P_g_goods_hat = np.zeros((data['n'], data['g']))
    C_g_hat_temp = C_g_hat ** (-data['theta_g'].reshape((1, data['g'])))
    part1 = np.sum(data['lambda_c'] * C_g_hat_temp.reshape((1, data['n'], data['g'])), axis=1)
    part2 = part1.reshape((data['n'], data['g']))
    P_g_goods_hat = part2 ** (-1/data['theta_g'].reshape((1, data['g'])))

    return P_g_goods_hat

In [27]:
syn_data['lambda_c'] = np.ones((3, 3, 4))
syn_data['theta_g'] =  np.linspace(1, 4, 4).reshape((4,1))*2

C_g_hat = np.ones((3, 4))*4

P_g_goods_hat = get_P_c_goods_hat(C_g_hat, syn_data)

In [28]:
P_g_goods_hat

array([[2.30940108, 3.03934274, 3.33073271, 3.48674217],
       [2.30940108, 3.03934274, 3.33073271, 3.48674217],
       [2.30940108, 3.03934274, 3.33073271, 3.48674217]])

In [35]:
## Demand 
# TODO: Check if they sort of work

In [36]:
def get_D_f_hat(E_hat, P_f_goods_hat, P_f_hat, data):
    ''' This is a function that returns the demand for final goods for a country.
    variables: E_hat, P_f_goods_hat, P_f_hat
    parameters: data['sigma']
    output: D_f_hat (nxk)'''
    
    D_f_hat = E_hat * (P_f_goods_hat/P_f_hat) ** (1 - data['sigma'])
    
    return D_f_hat

In [37]:
def get_D_c_hat(P_c_goods_hat, C_f_hat, Y_f_hat, data):
    ''' This is a function that returns the demand for commodities for a country.
    variables: P_c_goods_hat, C_f_hat, Y_f_hat
    parameters: data['d'], data['eta']
    output: D_c_hat (nxg)
    TODO: Problem with reshape
    '''
    C_f_hat_temp = C_f_hat **(data['eta'].reshape(1, data['k']) - 1)
    
    part_1 = data['d'] * ( C_f_hat_temp * Y_f_hat ).reshape((data['n'], data['k'], 1))
    part_2 = P_c_goods_hat.reshape((data['n'], 1, data['g'])) **(1 - data['eta'].reshape((1, data['k'], 1)))
    D_c_hat = np.sum(part_1*part_2, axis=1).reshape((data['n'], data['g']))
    
    return D_c_hat
    

In [41]:
syn_data['d'] = np.ones((3, 4, 4))*2
syn_data['eta'] = np.ones((syn_data['k'], 1))*4

In [42]:
## Production
# TODO: Check if they work

In [43]:
def get_Y_f_hat(C_f_hat, P_f_hat, D_f_hat, data):
    ''' This is a function that returns the total production for a given final good and country.
    variables: C_f_hat, P_f_hat, D_f_hat
    parameters: data['X_f'], data['Y_f'], data['theta_k']'''
    
    C_f_hat_temp = C_f_hat **(-data['theta_k'].reshape((1, data['k'])))
    P_f_hat_temp = P_f_hat **(data['theta_k'].reshape((1, data['k'])))
    
    part_1 = data['X_f'] * (P_f_hat_temp * D_f_hat).reshape((data['n'],1,data['k']))
    
    Y_f_hat = C_f_hat_temp/data['Y_f'] * np.sum(part_1, axis=1).reshape(data['n'], data['k'])
    
    return Y_f_hat

In [44]:
def get_Y_c_hat(C_c_hat, P_c_hat, D_c_hat, data):    
    ''' This is a function that returns the total production for a given final good and country.
    variables: C_c_hat, P_c_hat, D_c_hat
    parameters: data['X_c'], data['Y_c'], data['theta_g']'''
    
    C_c_hat_temp = C_c_hat **(-data['theta_g'].reshape((1, data['g'])))
    P_c_hat_temp = P_c_hat **(data['theta_g'].reshape((1, data['g'])))
    
    part_1 = data['X_c'] * (P_c_hat_temp * D_c_hat).reshape((data['n'],1,data['g']))
    
    Y_c_hat = C_c_hat_temp/data['Y_c'] * np.sum(part_1, axis=1).reshape(data['n'], data['g'])
    
    return Y_c_hat

In [47]:
## Cost of production
# TODO: Check if they are working

In [ ]:
def get_C_f_hat(w_hat, P_c_goods_hat, D_f_hat, data):
    ''' This is a function that returns the cost of final good in a country.
    variables: w_hat, P_c_goods_hat, D_f_hat
    parameters: data['phi_L_k'], data['phi'], data['eta'] '''
    
    part1 = data['phi_L_k'] * (w_hat.reshape((data['n'], 1)) ** (1 - data['eta'].reshape((1, data['k']))))
    part2 = data['phi'] * ((P_c_hat_goods.reshape(data['n'], 1, data['k'])) ** (1 - data['eta'].reshape((1, data['k'],1))) )
    
    C_f_hat = (part1 + np.sum(part2, axis=2).reshape((data['n'], data['k']))) ** (1/(1-data['eta'].reshape((1, data['k'])) ))
    
    return C_f_hat

In [ ]:
def get_C_c_hat(w_hat, r_hat, data):
    ''' This is a function that returns the cost of Commodities in a country.
    variables: w_hat, r_hat
    parameters: data['phi_L_g'], data['phi_R'], data['rho_g']
    '''
    
    part1 = data['phi_R'] * (r_hat ** (1- data['rho_g'].reshape((1, data['g']))))
    part2 = data['phi_L_g'] * (w_hat.reshape((data['n'], 1)) ** (1 - data['rho_g'].reshape((1, data['g']))))
    
    C_c_hat = (part1 + part2) ** (1/(1-data['rho_g'].reshape((1, data['g'])) ))
    
    return C_c_hat

In [ ]:
## Cost of production
# TODO: Write the equations out
# TODO: Check if they are working

In [48]:
def get_r_hat(Y_c_hat, C_c_hat, data):
    ''' This is a function that returns the ...
    variables: Y_c_hat, C_c_hat
    parameters: data['R_hat'], data['rho_g']'''
    
    part1 = (Y_c_hat/data['R_hat']) ** (1/data['rho_g'].reshape((1, data['g'])))
    part2 = (C_c_hat) ** (1 - 1/data['rho_g'].reshape((1, data['g'])))

    r_hat = part1*part2
    
    return r_hat

In [49]:
def get_w_hat(C_c_hat, C_f_hat, Y_c_hat, Y_f_hat, data):
    ''' This is a function that returns change in wage in a given country
    variables: C_c_hat, Y_c_hat, C_f_hat, Y_f_hat
    parameters: data['phi_L_g], data['phi_L_k], data['Y_c'], data['Y_f'], data['']'''
    
    '''I will not define this here as it has many w_hats in the equation itself. I can define this when I am 
    defining the equlibirium. It will be the only complicated equation I will state.'''
    return None